In [1]:
import getpass
import os

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../documents/sajith.txt")
docs = loader.load()

len(docs[0].page_content)

82985

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

107

In [5]:
len(all_splits[0].page_content)

815

In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents=all_splits, embedding=OpenAIEmbeddings(), persist_directory="./sajith_vectorstore"
)

In [7]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are the approaches to solve poverty?")

len(retrieved_docs)

6

In [8]:
print(retrieved_docs[0].page_content)

•Rs. 10,000/ month (from 5,000) for vulnerable and transitional poor families • Rs. 10,000/ month (from 8,500) for poor families • No changes to Rs. 15,000/ month for extremely poor families
This initiative will empower beneficiary families and communities by enhancing their capacities and skills, featuring components of consumption, savings, investment, production, and exports. At its core, this programme will implement a credit-based participatory poverty alleviation strategy through a robust social protection framework.
Recognizing the tremendous impact of the Samurdhi programme in eradicating poverty in Sri Lanka, a fresh scientific
  34
•
 •


In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What do you do about education?"):
    print(chunk, end="", flush=True)

The plan includes establishing trust funds for school sustainability, implementing a STEEAM education system from January 2025, and simplifying the Grade 5 Scholarship Examination to reduce mental stress on students. It also focuses on transforming schools into smart institutions with digital learning capabilities and filling teacher vacancies for religious education. Additionally, expert opinions will be sought to update curricula considering modern technological trends. Thanks for asking!